# [Chapter 4] Cleaning the global organ donation trends data

[DSLC stages]: Data cleaning and pre-processing


## Domain problem formulation

Our goal for this project is to understand global organ donation trends, in particular, we want to identify countries that have demonstrated an increase in organ transplant donation rates over time, and which countries have the highest organ donation rates. 


## Data source overview

In this file, we will examine and clean the organ donation data which comes from the publicly available survey data from the Global Observatory on Donation and Transplantation (GODT) that was collected in a collaboration between World Health Organization (WHO) and the Spanish Transplant Organization, Organización Nacional de Trasplantes (ONT). The data portal can be found at http://www.transplant-observatory.org/export-database/. 

This database contains information about organ donations and transplants (a total of 24 variables) for 194 countries and is collected every year based on a survey that, according to the website, began in 2007. We will consider a version of the data that contains information up to 2017 (downloaded in 2018) for this project.


First, let's load the libraries that we will use in this document.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)

The data is contained within the file `data/global-organ-donation_2018.csv`. The questionnaire on which the data is based is contained within the `data/documentation/Questionnaire.pdf` PDF file. If you plan on working with this data, we highly recommend that you glance through this document. 




## Step 1: Review background information

For additional background information on this project and dataset and the project, see the relevant PCS documentation for this project. 

- *What does each variable measure?* A data dictionary is presented in the subsection below. 

- *How was the data collected?* The website ([http://www.transplant-observatory.org/methodology/](http://www.transplant-observatory.org/methodology/)) from which we downloaded the organ donation data states that the data is based on a survey that began in 2007, and is sent annually via email to "national focal points". For countries that have a centralized organ donation and transplantation organization, this information would likely be much easier to obtain than for the countries that do not have well-organized transplant systems (or which have multiple donor organizations). A copy of the survey itself is provided in the "data/documentation" folder^[This survey was originally downloaded from http://www.transplant-observatory.org/questionnaire-pdf/]. Some questions that arise include: Who were these survey forms sent to (e.g., who is the contact point)? Which countries each country have a centralized organ donation organization? Some reading online revealed a very wide range of organ donation practices worldwide. 

- *What are the observational units?* To identify the observational units, consider for what entities each full set of organ donation measurements is collected. For the organ donation data, these are the year and country combinations (a complete set of measurements are taken for each country, every year), which we will call the "country-years".

- *Is the data relevant to my project?* Since this data is likely the most comprehensive public global summary of organ donations available, and since it covers a reasonably broad time period, this data is certainly relevant to the project.

- *What questions do I have and what assumptions am I making?* One immediate *assumption* that we made when looking at the data was that there exists a hierarchy for some of the variables. For instance, the total number of deceased donors (`TOTAL Actual DD`) appears to be broken down into brain-death deceased donors (`Actual DBD`) and circulatory-death deceased donors (`Actual DCD`), implying that these two sub-counts *should* add up to the total count. After we loaded the data into R (in the next step), we conducted some quick checks in the data to confirm that this is true in all but a small number of rare cases.



### Data dictionary

The data dictionary we found on the website at the time of data collection is printed below:



Some questions that immediately arise include what does it mean for an organ donor to be *"utilized"*? Does this imply that not all donated organs are used? After much scouring GODT resources, we eventually found this definition in the following pdf (https://tts.org/images/GODT/2020-Global-report-para-web-1.pdf):

- "Actual deceased donor": Deceased person from whom at least one organ has been recovered for the purpose of transplantation.

- "Utilized deceased donor": An actual donor from whom at least one organ has been transplanted. 


That is, an organ from an "actual deceased donor" has been recovered for the purpose of transplantation, but may not actually end up being transplanted (presumably due to logistical issues, such as not finding a suitable recipient, spoilage, etc). 

Our gut feeling is that we should use the `TOTAL Actual DD` variable, rather than the `Total Utilized DD` variable, but it will be helpful to identify how similar/different these two variables are:

> **Question: What proportion of deceased donors are "utilized" deceased donors?**
> 
> Do most organs recovered for transplantation actually get transplanted?


We will answer this question in our explorations below.

Feel free to document here any additional questions or assumptions that you made while looking at the website or the data dictionary above. 


## Step 2: Loading in the data


Let's load in the data. 


In [2]:
organs_original = pd.read_csv("../data/global-organ-donation_2018.csv") 

Below, we print the data column names and notice that they match the names presented in the data dictionary. 

In [3]:
organs_original.columns

Index(['REGION', 'COUNTRY', 'REPORTYEAR', 'POPULATION', 'TOTAL Actual DD',
       'Actual DBD', 'Actual DCD', 'Total Utilized DD', 'Utilized DBD',
       'Utilized DCD', 'DD Kidney Tx', 'LD Kidney Tx', 'TOTAL Kidney Tx',
       'DD Liver Tx', 'DOMINO Liver Tx', 'LD Liver Tx', 'TOTAL Liver TX',
       'Total Heart', 'DD Lung Tx', 'LD Lung Tx', 'TOTAL Lung Tx',
       'Pancreas Tx', 'Kidney Pancreas Tx', 'Small Bowel Tx'],
      dtype='object')


Below, we have printed the first 20 rows of the dataset. Note that *all of the values after the first 4 identifier columns are missing for these first 20 rows*. We checked the data manually to make sure that this was not a data loading error, and it does seem that the data has been loaded in correctly.


In [4]:
organs_original.head(20)

,REGION,COUNTRY,REPORTYEAR,POPULATION,TOTAL Actual DD,Actual DBD,Actual DCD,Total Utilized DD,Utilized DBD,Utilized DCD,DD Kidney Tx,LD Kidney Tx,TOTAL Kidney Tx,DD Liver Tx,DOMINO Liver Tx,LD Liver Tx,TOTAL Liver TX,Total Heart,DD Lung Tx,LD Lung Tx,TOTAL Lung Tx,Pancreas Tx,Kidney Pancreas Tx,Small Bowel Tx
0,Europe,Andorra,2000,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eastern Mediterranean,United Arab Emirates,2000,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eastern Mediterranean,Afghanistan,2000,22.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,America,Antigua and Barbuda,2000,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Europe,Albania,2000,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Europe,Armenia,2000,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Africa,Angola,2000,12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,America,Argentina,2000,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Europe,Austria,2000,8.2,194.0,194.0,0.0,NaN,NaN,NaN,357.0,37.0,394.0,151.0,NaN,NaN,151.0,87.0,NaN,NaN,59.0,30.0,30.0,6.0
9,Western Pacific,Australia,2000,18.9,195.0,195.0,NaN,NaN,NaN,NaN,350.0,178.0,528.0,149.0,NaN,NaN,149.0,57.0,NaN,NaN,90.0,26.0,26.0,NaN


We also print a *random* sample of 20 rows from the data below.

In [5]:
organs_original.sample(20)

,REGION,COUNTRY,REPORTYEAR,POPULATION,TOTAL Actual DD,Actual DBD,Actual DCD,Total Utilized DD,Utilized DBD,Utilized DCD,DD Kidney Tx,LD Kidney Tx,TOTAL Kidney Tx,DD Liver Tx,DOMINO Liver Tx,LD Liver Tx,TOTAL Liver TX,Total Heart,DD Lung Tx,LD Lung Tx,TOTAL Lung Tx,Pancreas Tx,Kidney Pancreas Tx,Small Bowel Tx
1738,America,Venezuela (Bolivarian Republic of),2008,28.1,93.0,NaN,0.0,NaN,NaN,NaN,178.0,100.0,278.0,2.0,0.0,8.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
779,America,Antigua and Barbuda,2004,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Europe,Denmark,2000,5.3,68.0,68.0,NaN,NaN,NaN,NaN,121.0,33.0,154.0,24.0,NaN,NaN,24.0,28.0,NaN,NaN,31.0,NaN,NaN,NaN
145,Eastern Mediterranean,Qatar,2000,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2919,Europe,Bulgaria,2015,7.1,45.0,45.0,0.0,19.0,19.0,0.0,38.0,10.0,48.0,15.0,0.0,1.0,16.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
198,Europe,Albania,2001,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,Eastern Mediterranean,Oman,2005,2.6,0.0,NaN,NaN,NaN,NaN,NaN,0.0,16.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,South-East Asia,Indonesia,2003,219.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2311,America,Trinidad and Tobago,2011,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,America,Dominican Republic,2000,8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, we check the dimension of the data.

In [6]:
organs_original.shape

(3165, 24)

The above code shows that there are 3,165 rows. As a sanity check, this number should probably be divisible by the number of countries in the data. The number of countries in the data is:

In [7]:
len(organs_original["COUNTRY"].unique())

194

But 3,165 is *not* divisible by 194:

In [8]:
3165 / 194

16.314432989690722

Moreover, if the survey started in 2007 and we have data up to the year 2017 (11 years total), then we should have $11 \times 194 = 2,134$ rows in the data. Clearly, *something* is wrong. At this stage, we don't know what is wrong but will make a note to ensure that we figure out what is going on. If by the end of the evaluations that we will conduct below, we haven't figured it out, then we will do some specific explorations to try and understand why.

> **Question: Why does the number of rows in the data not match what we expect?**
> 
> The number of rows in the data are not divisible by the number of countries, which is a bit strange, since we would have assumed that each country would have contributed the same number of rows to the data.




To determine the ways in which the data needs to be cleaned, we will follow the suggestions provided in Chapter 5 of Veridical Data Science.




## Step 3: Examine the data and create action items


In this section, we will look at the data itself to try to identify any invalid values, understand the missing values, and any abnormalities in the data, following the workflow outlined in Chapter 5.




### Finding invalid values



Below, we print out a summary of the values of each numeric column.


In [9]:
organs_original.select_dtypes('number').describe()

,REPORTYEAR,POPULATION,TOTAL Actual DD,Actual DBD,Actual DCD,Total Utilized DD,Utilized DBD,Utilized DCD,DD Kidney Tx,LD Kidney Tx,TOTAL Kidney Tx,DD Liver Tx,DOMINO Liver Tx,LD Liver Tx,TOTAL Liver TX,Total Heart,DD Lung Tx,LD Lung Tx,TOTAL Lung Tx,Pancreas Tx,Kidney Pancreas Tx,Small Bowel Tx
count,3165.000000,3165.000000,1290.000000,1181.000000,959.000000,492.000000,514.000000,525.000000,1308.000000,1370.000000,1401.000000,1217.000000,885.000000,1100.000000,1253.000000,1139.000000,592.000000,581.000000,1017.000000,921.000000,1053.000000,910.000000
mean,2007.710269,35.237283,310.726357,289.475868,38.872784,285.699187,248.227626,39.636190,487.107034,303.140876,752.231263,243.136401,1.447458,45.695455,277.059856,78.051800,56.121622,0.203098,59.720747,41.018458,29.842355,3.720879
std,4.793700,131.945656,1002.027550,922.151429,197.115786,972.149942,809.979679,223.237394,1430.753024,829.967985,2049.562922,772.811183,5.755855,140.703985,807.568306,283.838158,232.324876,1.710028,216.100281,163.899852,111.359410,20.060555
min,2000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2004.000000,1.900000,5.000000,4.000000,0.000000,0.000000,1.000000,0.000000,8.000000,12.250000,43.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2008.000000,7.400000,58.000000,53.000000,0.000000,28.000000,33.500000,0.000000,95.500000,51.000000,163.000000,30.000000,0.000000,2.000000,35.000000,11.000000,0.000000,0.000000,3.000000,3.000000,2.000000,0.000000
75%,2012.000000,23.600000,194.750000,182.000000,2.000000,161.000000,150.750000,0.000000,346.000000,190.000000,494.000000,138.000000,0.000000,18.000000,178.000000,53.500000,30.000000,0.000000,36.000000,24.000000,17.000000,0.000000
max,2017.000000,1393.800000,10286.000000,8403.000000,3298.000000,9706.000000,8075.000000,3243.000000,14827.000000,6647.000000,20638.000000,7715.000000,71.000000,1200.000000,8082.000000,3273.000000,2478.000000,20.000000,2478.000000,1484.000000,924.000000,198.000000


There don't appear to be any negative count values, but the population seems to be on a strange scale (why are there countries whose population is 0, why is the largest population just 1393.8, and how can you have .8 of a person?).

The table below shows the average recorded population for a sample of 20 countries.


In [10]:
organs_original.groupby("COUNTRY")["POPULATION"] \
    .mean() \
    .sample(20)

COUNTRY
Guyana                                         0.780000
Zimbabwe                                      13.060000
Grenada                                        0.100000
Bosnia and Herzegovina                         3.911765
The former Yugoslav Republic of Macedonia      2.055556
Gambia                                         1.613333
Argentina                                     40.250000
Yemen                                         22.233333
Comoros                                        0.766667
Slovakia                                       5.422222
Uzbekistan                                    27.243750
Madagascar                                    19.426667
Monaco                                         0.000000
Tunisia                                       10.312500
Belgium                                       10.688889
Niger                                         14.406667
Venezuela (Bolivarian Republic of)            27.800000
Costa Rica                              

Since we know the population of Australia should be more like 21.5 million, rather than 21.5, this implies that the populations are on a scale of millions. We manually checked that this is also true for a few other countries.

So that the population variable is as transparent as possible, we will note a cleaning action item to be included in our final cleaning function.


> **Data cleaning action item: Multiply the population variable by 1 million**
>
> Multiply the population variable by 1 million. We won't worry about the rounding error, but note that these population values are far from exact.




Taking another look at the first 20 rows printed in the table above (in the data loading section), we also notice that the year pre-dates 2007, which was when the survey supposedly began.


> **Question: Why does the data contain years pre-2007**
> 
> The data contains information prior to 2007, which was when the data collection survey supposedly began. Why is this the case? We couldn't find information online to answer this question, but perhaps this is due to back-reporting (i.e. countries providing historical data), or perhaps it is a mistake in the documentation.


> **Data cleaning action item: Add an option to remove the pre-2007 data**
>
> We want to have an option in our cleaning function to remove the pre-2007 data, but the default option will be to keep it. 



To investigate if there are any strange values in the `TOTAL Actual DD` variable, the figure below displays a histogram of the `TOTAL Actual DD` variable. 

In [11]:
px.histogram(organs_original, x="TOTAL Actual DD")

The distribution is heavily skewed with a lot of 0s, but nothing looks particularly unusual.

Before moving on, let's conduct some sanity checks by ensuring that the number of total donors does not exceed the population and is not exceeded by any of the sub-counts. The code below counts the number of times (rows in which) the total donor count exceeds the population and the number of times the total donor count is exceeded by the relevant sub-count. We would expect each of these sums to be equal to 0.


In [12]:
sum(organs_original["TOTAL Actual DD"] > organs_original["POPULATION"] * 1000000)


0

In [13]:
sum(organs_original["TOTAL Actual DD"] < organs_original["Actual DCD"])

0

In [14]:
sum(organs_original["TOTAL Actual DD"] < organs_original["Actual DBD"])

0

In [15]:
sum(organs_original["TOTAL Actual DD"] < organs_original["Total Utilized DD"])

0

In [16]:
sum(organs_original["TOTAL Actual DD"] < organs_original["TOTAL Kidney Tx"])

1201

In [17]:
sum(organs_original["TOTAL Actual DD"] < organs_original["TOTAL Liver TX"])

179

In [18]:
sum(organs_original["TOTAL Actual DD"] < organs_original["Total Heart"])

0

Interestingly, the number of kidney and liver transplant counts seem to often exceed the total number of donors. But using our domain knowledge, this is not actually surprising once we realize that a single donor can provide *two* kidneys for transplant, and a single donated liver can be split between two recipients.



Overall, the only unusual or inconsistent values that we found come from the population variable and the pre-2007 values. 



### [problem-specific] Checking COUNTRY and YEAR combinations are unique

It occurs to us to check that each country-year combination is unique (i.e., there are no duplicated entries in the data). The table below counts the number of times each country-year combination appears in the data and shows the distinct counts. Since the only entry is 1, this implies that each country-year combination only appears once in the data, which is good.

In [19]:
organs_original[["REPORTYEAR", "COUNTRY"]].value_counts().unique()

array([1])

### Examining missing values

From our tables and histograms above, it doesn't look like there are any extreme values (such as `999`) that are hiding missing values. To make sure, we will plot a histogram of a few of the variables.

The output below shows the number and proportion of missing rows in each column, arranged in order of least to most missingness.



In [20]:
missing_prop = organs_original.isna().sum() / len(organs_original.index)
missing_prop.sort_values()

REGION                0.000000
COUNTRY               0.000000
REPORTYEAR            0.000000
POPULATION            0.000000
TOTAL Kidney Tx       0.557346
LD Kidney Tx          0.567141
DD Kidney Tx          0.586730
TOTAL Actual DD       0.592417
TOTAL Liver TX        0.604107
DD Liver Tx           0.615482
Actual DBD            0.626856
Total Heart           0.640126
LD Liver Tx           0.652449
Kidney Pancreas Tx    0.667299
TOTAL Lung Tx         0.678673
Actual DCD            0.696998
Pancreas Tx           0.709005
Small Bowel Tx        0.712480
DOMINO Liver Tx       0.720379
DD Lung Tx            0.812954
LD Lung Tx            0.816430
Utilized DCD          0.834123
Utilized DBD          0.837599
Total Utilized DD     0.844550
dtype: float64

Other than the descriptor and ID variables of `REGION`, `COUNTRY`, `REPORTYEAR`, and `POPULATION`, there are a LOT of missing values in this dataset. Almost 60% of the `TOTAL Actual DD` values (our variable of interest) are missing!

The figure below shows the distribution of missing data across the entire dataset.


In [21]:
px.imshow(organs_original.isna().astype(int), 
          color_continuous_scale='Greys')


How this missingness is distributed over time? The figure below shows the number of *non-missing* values reported for each year. Clearly, the earlier years (especially pre-2007!) have more missing values than the later years, but the missingness starts to increase again after 2014, which seems odd. 

In [22]:
missing_values_by_year = (organs_original.set_index("REPORTYEAR")["TOTAL Actual DD"] # set the index to be REPORTYEAR and select just TOTAL Actual DD col
                                         .notna() # identify whether each TOTAL Actual DD value is not missing
                                         .groupby(level=0) # group by REPORTYEAR (index)
                                         .sum()) # add up the total number of missing TOTAL Actual DD values each year
missing_values_by_year

px.bar(missing_values_by_year, labels={"value": "Number of non-missing TOTAL Actual DD values"})

What about the distribution of missingness by country? Is it the case that there are countries for which individual years of data are missing? Or is it instead that all of the data are missing or none of it is?

The table below shows the number of non-missing `TOTAL Actual DD` entries for a random sample of 20 countries. There seems to be a really big range of missingness patterns across the countries! Some countries report literally no non-missing data, whereas others report data for 7, 10, or 13 years.


In [23]:

# identify whether each TOTAL Actual DD value is not missing
# and group by COUNTRY (index)
# and add up the number of non-missing values
non_missing_by_country = organs_original.set_index("COUNTRY")["TOTAL Actual DD"] \
    .notna() \
    .groupby(level=0) \
    .sum() 

# sample 20 random countries (with a seed)
# and arrange countries alphabetically
non_missing_by_country.sample(20, random_state=1303) \
    .sort_index()


COUNTRY
Algeria                              7
Antigua and Barbuda                  0
Austria                             18
Bahamas                              0
Bolivia (Plurinational State of)     9
Comoros                              0
Cook Islands                         0
Dominican Republic                  13
Iran (Islamic Republic of)          17
Israel                              17
Latvia                              18
Malawi                               0
Myanmar                              8
Pakistan                            10
Portugal                            18
Samoa                                0
Sierra Leone                         0
Sudan                               11
Sweden                              18
Togo                                 0
Name: TOTAL Actual DD, dtype: int64

Let's visualize the distribution of non-missing data by country in the figure below

In [24]:
px.histogram(non_missing_by_country, 
             nbins=19, 
             labels={"value": "Number of non-missing TOTAL Actual DD values per country"})

It seems that almost 80 countries report absolutely no data (the first, left-most bar), while fewer than 20 countries report data every year (the final, right-most bar). That's quite disappointing... 

Out of curiosity, let's explicitly look at the data for some of the countries that report data every year, and some of the countries who report data for just *some* of the years.

The table below shows the `TOTAL Actual DD` counts for Austria, which had non-missing values for every year.


In [25]:
# filter to just austria
organs_austria = organs_original.query('COUNTRY == "Austria"')
# print out just the country, year, and total actual DD columns
organs_austria[["COUNTRY", "REPORTYEAR", "TOTAL Actual DD"]]

,COUNTRY,REPORTYEAR,TOTAL Actual DD
8,Austria,2000,194.0
202,Austria,2001,191.0
396,Austria,2002,195.0
590,Austria,2003,189.0
784,Austria,2004,185.0
978,Austria,2005,203.0
1172,Austria,2006,207.0
1366,Austria,2007,185.0
1560,Austria,2008,172.0
1754,Austria,2009,214.0



The table below, however, shows the results for Peru, which has data for 10 of the 18 years

In [26]:
# filter to just peru
organs_peru = organs_original.query('COUNTRY == "Peru"')
# print out just the country, year, and total actual DD columns
organs_peru[["COUNTRY", "REPORTYEAR", "TOTAL Actual DD"]]

,COUNTRY,REPORTYEAR,TOTAL Actual DD
137,Peru,2000,NaN
331,Peru,2001,NaN
525,Peru,2002,NaN
719,Peru,2003,NaN
913,Peru,2004,29.0
1107,Peru,2005,22.0
1301,Peru,2006,NaN
1495,Peru,2007,NaN
1689,Peru,2008,NaN
1883,Peru,2009,NaN


The type of missingness is interesting and definitely tells us something about how we might want to impute these values since it is unlikely that those middle missing values should be zero (i.e., it is unlikely that `NA` here represents a lack of any donations).

Depending on the analysis that we want to conduct, it will likely be helpful to impute the missing values (e.g., if we are trying to compute total donor counts over time to reduce the extent of under-counting). For some visualizations though, we won't need imputed values, so imputation will be an optional pre-processing action item.

Some reasonable action items for dealing with the missingness might be to replace each missing count for each country with:

- The *average* of the two surrounding non-missing values from the country.

- The *closest* (in terms of year) non-missing value from the country. If the missing value is equidistant between two non-missing years, we could choose one at random. 

- The *previous* non-missing value from the country.

- An *interpolated* value that takes into account the trend from all of the non-missing values from the country.

For the countries that do not report *any* donor counts (i.e., all their data is missing), we will impute their donor counts with 0. Here we are making an *assumption* that the countries that have entirely missing data do not have an organ donation system in place. This is probably not an entirely realistic assumption, however since there may be some countries that do have an organ donor system but chose not to report any data to the GODT, but since this data does not exist in the public domain, there is nothing else that we can really do about it.



> **Pre-processing action item: Impute the donor count variable**
> 
> There are several judgment call options that seem reasonable for creating imputed donor count variables, including:
> 
> - The *average* of the two surrounding non-missing values from the country.
>
> - The *previous* non-missing value from the country.
>
> For the countries that do not report *any* donor counts (i.e., all their data is missing), we will impute their donor counts with 0. 


Other options include imputing using the *closest* (in terms of year) non-missing value from the country and using an *interpolated* value that takes into account the trend from all of the non-missing values from the country, but these are much trickier to code so we exclude them here. 



#### Imputation function

Below, we define a function, `impute_feature()`, that we will later call inside our data cleaning/pre-processing function to impute the missing values, with arguments for each imputation judgment call option. 


In [27]:
def impute_feature(data, feature, group, impute_method="average"):
    impute_method = impute_method.lower()
    
    if impute_method == "previous":
        # create a new variable called feature_imputed that is equal to feature
        data = data.assign(feature_imputed=data[feature]) \
                            .groupby(group) \
                            .fillna(method='ffill') \
                            .fillna(0) # impute remaining missing values with 0
        return data["feature_imputed"]
    elif impute_method == "average":
        # create two temporary variables each equal to feature
        data = data.assign(imputed_feature_tmp_prev=data[feature], imputed_feature_tmp_next=data[feature])
        # fill the first variable using forward fill
        data["imputed_feature_tmp_prev"] = data.groupby(group)["imputed_feature_tmp_prev"].fillna(method='ffill')
        # fill the second variable using backward fill
        data["imputed_feature_tmp_next"] = data.groupby(group)["imputed_feature_tmp_next"].fillna(method='bfill')
        # then define feature_imputed column to be the mean of the forward and backward filled values
        data['feature_imputed'] = data[['imputed_feature_tmp_next', 'imputed_feature_tmp_prev']].mean(axis=1, skipna=True)
        # impute any remaining missing values with 0
        data['feature_imputed'].fillna(0, inplace=True)
        # remove the two temporary variables
        data = data.drop(columns=['imputed_feature_tmp_prev', 'imputed_feature_tmp_next'])
        return data["feature_imputed"]
    else:
        raise ValueError

We can compare the first 20 original and imputed TOTAL Actual DD donor counts using:


In [28]:
organs_original["imputed_donors"] = impute_feature(organs_original, 
                                                   feature = "TOTAL Actual DD", 
                                                   group = "COUNTRY", 
                                                   impute_method = "average")
organs_original[["COUNTRY", "REPORTYEAR", "TOTAL Actual DD", "imputed_donors"]].sample(20, random_state=1010)

/var/folders/k8/pvkv_0696v54cflkjhhkdlr00000gp/T/ipykernel_77462/2034971799.py:15: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/k8/pvkv_0696v54cflkjhhkdlr00000gp/T/ipykernel_77462/2034971799.py:17: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,COUNTRY,REPORTYEAR,TOTAL Actual DD,imputed_donors
1951,Bosnia and Herzegovina,2010,NaN,1.0
1779,CÃ´te dâ€™Ivoire,2009,NaN,0.0
3095,Syrian Arab Republic,2016,0.0,0.0
977,Argentina,2005,407.0,407.0
1592,Montenegro,2008,NaN,1.0
3100,South Africa,2016,72.0,72.0
600,Burundi,2003,NaN,0.0
270,Hungary,2001,137.0,137.0
214,Brunei Darussalam,2001,NaN,0.0
68,Equatorial Guinea,2000,NaN,0.0



### Assessing Column names


The column names in this dataset are a mess! Let's add an action item to clean them so that they are tidily formatted and human-readable.

> **Data cleaning action item: Clean the column names**
>
> Rename the columns so that they are consistently formatted, with underscore-separated words and human readable. Since we want to change the names of the variables themselves, we will do this manually.


The table below displays the column name conversion:

| New variable name | Original variable name |
| :---        |    :----   | 
| `region` | `REGION` |
| `country` | `COUNTRY` |
| `year` | `REPORTYEAR` |
| `population` | `POPULATION` |
| `total_deceased_donors` | `TOTAL Actual DD` |
| `deceased_donors_brain_death` | `Actual DBD` |
| `deceased_donors_circulatory_death` | `Actual DCD` |
| `total_utilized_deceased_donors` | `Total Utilized DD` |
| `utilized_deceased_donors_brain_death` | `Utilized DBD` |
| `utilized_deceased_donors_circulatory_death` | `Utilized DCD` |
| `deceased_kidney_tx` | `DD Kidney Tx` |
| `living_kidney_tx` | `LD Kidney Tx` |
| `total_kidney_tx` | `TOTAL Kidney Tx` |
| `deceased_liver_tx` | `DD Liver Tx` |
| `living_liver_tx` | `LD Liver Tx` |
| `domino_liver_tx` | `DOMINO Liver Tx` |
| `total_liver_tx` | `TOTAL Liver TX` |
| `total_heart_tx` | `Total Heart` |
| `deceased_lung_tx` | `DD Lung Tx` |
| `living_lung_tx` | `DD Lung Tx` |
| `total_lung_tx` | `TOTAL Lung Tx` |
| `total_pancreas_tx` | `Pancreas Tx` |
| `total_kidney_pancreas_tx` | `Kidney Pancreas Tx` |
| `total_small_bowel_tx` | `Small Bowel Tx` |


We will officially only change the column names when we actually clean our data below, so the remaining explorations until then will still use the original column names.




### Assessing variable type

The table below prints out the class/type of each column in the data.


In [29]:
organs_original.dtypes

REGION                 object
COUNTRY                object
REPORTYEAR              int64
POPULATION            float64
TOTAL Actual DD       float64
Actual DBD            float64
Actual DCD            float64
Total Utilized DD     float64
Utilized DBD          float64
Utilized DCD          float64
DD Kidney Tx          float64
LD Kidney Tx          float64
TOTAL Kidney Tx       float64
DD Liver Tx           float64
DOMINO Liver Tx       float64
LD Liver Tx           float64
TOTAL Liver TX        float64
Total Heart           float64
DD Lung Tx            float64
LD Lung Tx            float64
TOTAL Lung Tx         float64
Pancreas Tx           float64
Kidney Pancreas Tx    float64
Small Bowel Tx        float64
imputed_donors        float64
dtype: object

### [Exercise: to complete] Examining data completeness

While we checked that the ID variables (country and year combination) were *unique*, we didn't check whether all of the ID variables appear in the data (i.e., whether the data is *complete*). One way to check this is to confirm that every country has 18 rows in the data.

The table below shows the number of rows for each 20 randomly chosen countries. There are clearly NOT 18 rows for every country! In fact, there are almost never 18 rows for every country.

In [30]:
organs_original.value_counts("COUNTRY").sample(20, random_state=383)

COUNTRY
Lebanon         16
Kiribati        15
Zambia          15
Saudi Arabia    18
Cook Islands    15
Indonesia       15
Uzbekistan      16
Cyprus          18
Saint Lucia     15
Samoa           15
Sierra Leone    15
Serbia          16
Bahamas         15
Honduras        18
Iceland         18
Malawi          15
Netherlands     18
Armenia         18
India           16
Burundi         15
Name: count, dtype: int64

This is clearly related to our earlier question of "Why does the number of rows in the data not match what we expect?". It is left as an exercise to see if you can figure out what is going on. We recommend starting by comparing the number of non-missing `TOTAL Actual DD` values with the number of rows each year. Does something change around 2015?

To get you started, here is a bar chart counting the number of rows in the data for each year. 

In [31]:
px.histogram(organs_original, x="REPORTYEAR")

One exploration idea is to modify the code above to overlay another bar chart (using a different color) of the number of non-missing `TOTAL Actual DD` counts. 


> **Data cleaning action item: Complete the data**
> 
> Complete the data by adding in the missing rows and filling them with NA values. This should be done *prior* to missing value imputation so that the missing values we create are filled in during imputation.



Having concluded the suggested explorations, let's now address any remaining questions and assumptions that we have. 






### [problem-specific] Checking the hierarchy of donor count variables

Recall that we assumed that the `TOTAL Actual DD` donor count is made up of the sum of the `Actual DBD` (brain-death) and `Actual DCD` (circulatory death) donor counts. Let's check to see whether this is actually the case. 

If we ignore cases where there are missing values in any one of the three variables, then 99.3% of the remaining 945 rows satisfy the  `TOTAL Actual DD` = `Actual DBD` + `Actual DCD`:


In [32]:
# drop rows with missing values in any of the three columns
organs_no_missing = organs_original.dropna(subset=["TOTAL Actual DD", "Actual DBD", "Actual DCD"])
# compute the proportion of rows for which TOTAL Actual DD is equal to Actual DBD + Actual DCD
sum(organs_no_missing["TOTAL Actual DD"] == organs_no_missing["Actual DBD"] + organs_no_missing["Actual DCD"]) / len(organs_no_missing.index)

0.9925925925925926

(However, note that only 30% of the rows have non-missing values in all three variables.)


## Step 4: Clean and pre-process the data

Now we're ready to actually clean the data. Throughout our explorations above, we created the following action items (listed in the order they will be implemented):

- Rename the column names so that they are consistent, human-readable, underscore-separated, and lowercase

- Complete the data by adding in absent rows. The entries will be filled with missing values.

- Multiply the population variable by 1 million

We also created the following pre-processing action items (that are not necessary for the data to be clean, but they will be useful for our analyses):

- Add an imputed version of the `TOTAL Actual DD` count variable. There are several judgment call options for this step ("average", "previous", "closest", "interpolate"). We could also add imputed versions of the other variables too^[As an exercise, you may want to try and modify the `impute_feature()` code to also impute some of the transplant variables.], but we will focus on this variable in our analysis, we opted to just impute this variable (as well as the population variable values that were filled as missing when we completed the data).

- Add an option to remove (or not) the pre-2007 data. The default is to not remove this data.

To keep things simple, rather than writing a separate pre-processing function, we just wrote a single "data preparation" function that included both the cleaning and pre-processing action items:

We saved the cleaning function, `prepare_organ_data()` in the file `functions/prepare_organ_data.py`. This function makes use of the `impute_feature()` imputation function that we wrote which can be found in the `functions/impute_feature.py` file.  The `prepare_organ_data()` function is reproduced below.


In [33]:
import pandas as pd
from functions.impute_feature import impute_feature


def prepare_organ_data(organs_original,
                       impute_method = "average",
                       per_mil_vars = True): 
  
  
  # define a cleaned version of the original organs data
  # rename the original rows
  organs_clean = organs_original.rename(columns={
    'REGION': 'region',
    'COUNTRY': 'country',
    'REPORTYEAR': 'year',
    'POPULATION': 'population',
    'TOTAL Actual DD': 'total_deceased_donors',
    'Actual DBD': 'deceased_donors_brain_death',
    'Actual DCD': 'deceased_donors_circulatory_death',
    'Total Utilized DD': 'total_utilized_deceased_donors',
    'Utilized DBD': 'utilized_deceased_donors_brain_death',
    'Utilized DCD': 'utilized_deceased_donors_circulatory_death',
    'DD Kidney Tx': 'deceased_kidney_tx',
    'LD Kidney Tx': 'living_kidney_tx',
    'TOTAL Kidney Tx': 'total_kidney_tx',
    'DD Liver Tx': 'deceased_liver_tx',
    'LD Liver Tx': 'living_liver_tx',
    'DOMINO Liver Tx': 'domino_liver_tx',
    'TOTAL Liver TX': 'total_liver_tx',
    'Total Heart': 'total_heart_tx',
    'DD Lung Tx': 'deceased_lung_tx',
    'DD Lung Tx': 'living_lung_tx',
    'TOTAL Lung Tx': 'total_lung_tx',
    'Pancreas Tx': 'total_pancreas_tx',
    'Kidney Pancreas Tx': 'total_kidney_pancreas_tx',
    'Small Bowel Tx': 'total_small_bowel_tx'}).copy()

  # add the rows with missing country-year combinations
  country_year_combinations = pd.MultiIndex.from_product([organs_clean[col].unique() for col in ["country", "year"]], 
                                                         names=["country", "year"])
  # put these combinations into a data frame
  country_year_combinations_df = pd.DataFrame(index=country_year_combinations).reset_index()
  organs_clean = country_year_combinations_df.merge(organs_clean, on=["country", "year"], how="left")

  # For newly added rows, fill region with the unique values from the pre-existing rows  
  organs_clean["region"] = organs_clean.groupby("country")["region"].transform(lambda x: x.ffill().bfill())

  # multiply the population variable by 1 million
  organs_clean["population"] = organs_clean["population"] * 1000000
  
  # add imputed features using the specified imputation method
  # impute_feature() is a custom function defined in imputeFeature.R
  # Note that we are only imputing the total_deceased_donors variable and the 
  # population variable (missing values were introduced when we 
  # "completed" the data). You could impute more variables if you wanted to.
  if impute_method in ["average", "previous"]:
    organs_clean["population_imputed"] = impute_feature(organs_clean, 
                                                        feature = "population",
                                                        group = "country",
                                                        impute_method = impute_method)
    organs_clean["total_deceased_donors_imputed"] = impute_feature(organs_clean, 
                                                                  feature = "total_deceased_donors",
                                                                  group = "country",
                                                                  impute_method = impute_method)
  
  
  # rearrange the columns 
  column_order = ['country', 'year', 'region', 'population', 'population_imputed', 
                  'total_deceased_donors', 'total_deceased_donors_imputed'] + list(organs_clean.columns)
  column_order = pd.unique(column_order)
  organs_clean = organs_clean.reindex(columns=column_order)
  
  return organs_clean



### Checking the cleaned and pre-processed data

Let's do some exploration to ensure that the data was cleaned and pre-processed as expected.

Note: **to use the `prepare_organ_data()` function as in the code below, you will need to either run the code that defines the `prepare_organ_data()` and `impute_feature()` functions or you will need to source these files using the following code**:

In [34]:
# load the functions I wrote
from functions.prepare_organ_data import prepare_organ_data

In [35]:
# create the organs_clean object
organs_clean = prepare_organ_data(organs_original)

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:14: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:16: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:14: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:16: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a futur

Note the `TOTAL Actual DD` variable is now called `total_deceased_donors`, and we decided that the cleaned/pre-processed data should contain *both* an imputed version of this variable *in addition to* the original unimputed version (we would not want this if we were doing predictive modeling, say, but it is fine for exploratory projects such as this one). 

Here are the first 10 rows:


In [36]:
organs_clean.head(10)

,country,year,region,population,population_imputed,total_deceased_donors,total_deceased_donors_imputed,deceased_donors_brain_death,deceased_donors_circulatory_death,total_utilized_deceased_donors,utilized_deceased_donors_brain_death,utilized_deceased_donors_circulatory_death,deceased_kidney_tx,living_kidney_tx,total_kidney_tx,deceased_liver_tx,domino_liver_tx,living_liver_tx,total_liver_tx,total_heart_tx,living_lung_tx,LD Lung Tx,total_lung_tx,total_pancreas_tx,total_kidney_pancreas_tx,total_small_bowel_tx,imputed_donors
0,Andorra,2000,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Andorra,2001,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Andorra,2002,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Andorra,2003,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,Andorra,2004,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,Andorra,2005,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,Andorra,2006,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,Andorra,2007,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,Andorra,2008,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,Andorra,2009,Europe,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


and a random set of 10 rows:

In [37]:
organs_clean.sample(10)

,country,year,region,population,population_imputed,total_deceased_donors,total_deceased_donors_imputed,deceased_donors_brain_death,deceased_donors_circulatory_death,total_utilized_deceased_donors,utilized_deceased_donors_brain_death,utilized_deceased_donors_circulatory_death,deceased_kidney_tx,living_kidney_tx,total_kidney_tx,deceased_liver_tx,domino_liver_tx,living_liver_tx,total_liver_tx,total_heart_tx,living_lung_tx,LD Lung Tx,total_lung_tx,total_pancreas_tx,total_kidney_pancreas_tx,total_small_bowel_tx,imputed_donors
1651,Kiribati,2013,Western Pacific,100000.0,100000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1973,Monaco,2011,Europe,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
696,Colombia,2012,America,47600000.0,47600000.0,373.0,373.0,373.0,0.0,NaN,NaN,NaN,693.0,89.0,782.0,211.0,0.0,5.0,216.0,101.0,13.0,0.0,13.0,5.0,5.0,5.0,373.0
1309,Guyana,2013,America,800000.0,800000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
772,Cape Verde,2016,Africa,NaN,500000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,Brazil,2000,America,170100000.0,170100000.0,NaN,1408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1408.0
2271,Niger,2003,Africa,12000000.0,12000000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1346,Croatia,2014,Europe,4300000.0,4300000.0,151.0,151.0,151.0,0.0,NaN,NaN,NaN,186.0,11.0,197.0,124.0,0.0,1.0,125.0,34.0,0.0,0.0,0.0,5.0,4.0,0.0,151.0
2652,Serbia,2006,Europe,9900000.0,9900000.0,NaN,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0
2542,Poland,2004,Europe,38600000.0,38600000.0,562.0,562.0,562.0,NaN,NaN,NaN,NaN,1045.0,22.0,1067.0,181.0,NaN,18.0,199.0,105.0,NaN,NaN,NaN,16.0,15.0,NaN,562.0


The table below shows the relevant columns for the cleaned data for Peru. Note that the `TOTAL Actual DD` variable is now called `total_deceased_donors`, and the imputed version is called `total_deceased_donors_imputed`. 

In [38]:
organs_clean.query('country == "Peru"')[["country", "year", "population", "total_deceased_donors", "total_deceased_donors_imputed"]]

,country,year,population,total_deceased_donors,total_deceased_donors_imputed
2466,Peru,2000,25700000.0,NaN,29.0
2467,Peru,2001,26100000.0,NaN,29.0
2468,Peru,2002,26500000.0,NaN,29.0
2469,Peru,2003,27200000.0,NaN,29.0
2470,Peru,2004,27600000.0,29.0,29.0
2471,Peru,2005,28000000.0,22.0,22.0
2472,Peru,2006,28400000.0,NaN,58.0
2473,Peru,2007,28800000.0,NaN,58.0
2474,Peru,2008,28200000.0,NaN,58.0
2475,Peru,2009,29200000.0,NaN,58.0


Below, we also check that the "previous" imputation method works as expected for Peru:

In [39]:
organs_clean_previous = prepare_organ_data(organs_original, 
                                           impute_method = "previous")
organs_clean_previous.query('country == "Peru"')[["country", "year", "population", "total_deceased_donors", "total_deceased_donors_imputed"]]

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:7: FutureWarning:

DataFrameGroupBy.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:7: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:7: FutureWarning:

DataFrameGroupBy.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/u6047033/Documents/book/vds-book-supplementary/python/organ_donations/dslc_documentation/functions/impute_feature.py:7: FutureWarning:

DataFrame.fillna with 'method' is deprecated and

,country,year,population,total_deceased_donors,total_deceased_donors_imputed
2466,Peru,2000,25700000.0,NaN,0.0
2467,Peru,2001,26100000.0,NaN,0.0
2468,Peru,2002,26500000.0,NaN,0.0
2469,Peru,2003,27200000.0,NaN,0.0
2470,Peru,2004,27600000.0,29.0,29.0
2471,Peru,2005,28000000.0,22.0,22.0
2472,Peru,2006,28400000.0,NaN,22.0
2473,Peru,2007,28800000.0,NaN,22.0
2474,Peru,2008,28200000.0,NaN,22.0
2475,Peru,2009,29200000.0,NaN,22.0
